<a href="https://colab.research.google.com/github/py811/RLSS2022_AI_and_COVID/blob/main/correlation_covid%26weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries and csv file

In [ ]:
#importing useful libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy.stats as s
from scipy.stats.mstats import spearmanr,kendalltau
from datetime import datetime, date, timedelta
import math

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#import pingouin as pg


In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
#read COVID-19 data 
Covid_Data = pd.read_csv('https://raw.githubusercontent.com/py811/RLSS2022_AI_and_COVID/main/COVID_DATA.csv',usecols=[0,2,3,4,8,17,18,19,20,21])
#Covid_Data = pd.read_csv('merged_df.csv',usecols=[0,2,3,4,8,17,18,19,20,21])
Covid_Data.rename(columns={'Difference_Yesterday':'PCR_Postive_Count','Hospitalized_diff':'PCR_Positive_Hospitalised','Death_diff':'Deceased'}, inplace = True)
#data.drop([0,1], axis=0, inplace=True)
Covid_Data['PCR_Positive_Hospitalised'] = Covid_Data['PCR_Positive_Hospitalised'].fillna(0).astype('int64')
#Covid_Data

In [ ]:
Covid_Data1 = Covid_Data
Covid_Data1['Date']=pd.to_datetime(Covid_Data['Date'].astype(str), format='%d-%m-%Y')
Covid_Data1



,Date,District,Total_Confirmed,PCR_Postive_Count,Current_Infected,PCR_Positive_Hospitalised,Deceased,temperature,wind_speed,relative_humidity
0,2021-08-02,Altenkirchen,4961,5,26,0,0,14.892,7.688,85.36
1,2021-08-02,Alzey-Worms,5131,1,36,1,0,15.268,6.860,85.24
2,2021-08-02,Bad Kreuznach,6127,6,59,0,0,16.572,6.860,78.96
3,2021-08-02,Bernkastel-Wittlich,3121,0,57,0,0,14.060,8.804,82.88
4,2021-08-02,Birkenfeld,3461,0,64,0,0,14.208,6.352,88.88
...,...,...,...,...,...,...,...,...,...,...
9985,2022-07-29,KS Neustadt a.d.W.,17874,45,1512,0,0,21.292,15.624,51.80
9986,2022-07-29,KS Pirmasens,12073,62,945,0,0,21.136,10.332,52.24
9987,2022-07-29,KS Speyer,19090,47,1537,0,0,23.992,7.548,52.44
9988,2022-07-29,KS Trier,34768,46,2592,1,0,21.864,8.104,49.56


In [ ]:
import plotly.express as px


data = Covid_Data1.copy()

fig = px.line(data, x="Date", y="PCR_Postive_Count", color='District')

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))
fig.show()



In [ ]:
Covid_Data1.corr(method='spearman')


,Total_Confirmed,PCR_Postive_Count,Current_Infected,PCR_Positive_Hospitalised,Deceased,temperature,wind_speed,relative_humidity
Total_Confirmed,1.000000,0.665910,0.838679,0.300484,0.105375,0.175932,-0.001524,-0.539052
PCR_Postive_Count,0.665910,1.000000,0.735633,0.465192,0.197366,-0.073222,0.066340,-0.327354
Current_Infected,0.838679,0.735633,1.000000,0.306257,0.141908,-0.184357,0.121264,-0.434131
PCR_Positive_Hospitalised,0.300484,0.465192,0.306257,1.000000,0.260411,-0.041966,0.020671,-0.083114
Deceased,0.105375,0.197366,0.141908,0.260411,1.000000,-0.090341,0.036405,0.001910
temperature,0.175932,-0.073222,-0.184357,-0.041966,-0.090341,1.000000,-0.105699,-0.488885
wind_speed,-0.001524,0.066340,0.121264,0.020671,0.036405,-0.105699,1.000000,-0.060486
relative_humidity,-0.539052,-0.327354,-0.434131,-0.083114,0.001910,-0.488885,-0.060486,1.000000


In [ ]:
#Aggregating the Covid Data for the whole state RLP
Covid_Data_avg = Covid_Data1.groupby(['Date']).agg({'PCR_Postive_Count' : 'sum', 'PCR_Positive_Hospitalised' : 'sum',
                                                     'Deceased' : 'sum','Total_Confirmed' : 'sum','Current_Infected' : 'sum',
                                             'temperature' : 'mean', 'wind_speed' : 'mean', 'relative_humidity': 'mean',
                                            }).reset_index()
#Covid_Data_avg.to_csv('Covid_Data_avg.csv')
Covid_Data_avg['Day_of_week'] = Covid_Data_avg['Date'].dt.day_name()
Covid_Data_avg

,Date,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,Day_of_week
0,2021-08-02,122,3,0,138639,1560,15.408933,10.650933,83.221333,Monday
1,2021-08-03,142,8,0,138781,1646,14.946933,10.774667,87.574667,Tuesday
2,2021-08-04,170,6,1,138951,1758,16.023067,7.270933,81.329333,Wednesday
3,2021-08-05,129,1,0,139080,1830,17.316933,9.500400,80.693333,Thursday
4,2021-08-06,156,10,3,139236,1901,16.960667,16.658667,80.053333,Friday
...,...,...,...,...,...,...,...,...,...,...
328,2022-07-25,6598,63,14,1184097,84698,24.600400,15.073067,51.744000,Monday
329,2022-07-26,5792,137,6,1189889,89989,20.124400,12.751867,58.258667,Tuesday
330,2022-07-27,4841,104,3,1194730,94494,18.504533,12.537733,54.682667,Wednesday
331,2022-07-28,3913,76,7,1198643,92589,20.134000,12.251200,49.390667,Thursday


In [ ]:
Covid_Data_avg.describe()

,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity
count,333.000000,333.000000,333.000000,3.330000e+02,333.000000,333.000000,333.000000,333.000000
mean,3192.078078,42.783784,5.159159,4.746681e+05,61158.579580,10.138873,12.775438,75.616134
std,3376.401219,31.840833,5.045744,3.567498e+05,59781.913739,6.576261,5.637042,13.521091
min,47.000000,-2.000000,-1.000000,1.386390e+05,1560.000000,-3.597733,4.945733,35.965333
25%,479.000000,18.000000,1.000000,1.668710e+05,9911.000000,4.616533,9.111467,65.677333
50%,1895.000000,36.000000,4.000000,2.698110e+05,32849.000000,9.313867,11.298000,79.716000
75%,4932.000000,63.000000,8.000000,8.431730e+05,103331.000000,15.840400,15.124800,86.692000
max,14163.000000,137.000000,24.000000,1.201776e+06,213112.000000,27.189200,36.224800,96.386667


In [ ]:
data = Covid_Data_avg.copy()
# fig = make_subplots(specs=[[{"secondary_y": True}]])
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Total_Confirmed'],name="Total_Confirmed", mode='lines',line_color = 'green'))

#fig.add_trace(
#    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
#    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black',
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Total_Confirmed", title_font_color= 'green', 
                showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                showline=True, linewidth=1.5, linecolor='green',
                showticklabels = True, ticks="inside", tickfont_color = 'green')
#, range = [0,14500])

#fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
#                 showline=True, linewidth=1.5, linecolor='royalblue',
#                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_layout(plot_bgcolor="white")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/PCR_Temp.png")

In [ ]:
data = Covid_Data_avg.copy()
# fig = make_subplots(specs=[[{"secondary_y": True}]])
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'))

#fig.add_trace(
#    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
#    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black',
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green', 
                showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                showline=True, linewidth=1.5, linecolor='green',
                showticklabels = True, ticks="inside", tickfont_color = 'green')
#, range = [0,14500])

#fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
#                 showline=True, linewidth=1.5, linecolor='royalblue',
#                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_layout(plot_bgcolor="white")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/PCR_Temp.png")

In [ ]:
Covid_Data_avg.corr(method='spearman')

,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity
PCR_Postive_Count,1.000000,0.821788,0.563022,0.784037,0.859562,-0.148613,0.190590,-0.431754
PCR_Positive_Hospitalised,0.821788,1.000000,0.738336,0.569711,0.601362,-0.079667,0.105082,-0.254196
Deceased,0.563022,0.738336,1.000000,0.325752,0.385715,-0.243298,0.067732,0.006976
Total_Confirmed,0.784037,0.569711,0.325752,1.000000,0.818746,0.090358,0.118649,-0.552613
Current_Infected,0.859562,0.601362,0.385715,0.818746,1.000000,-0.206005,0.216336,-0.496328
temperature,-0.148613,-0.079667,-0.243298,0.090358,-0.206005,1.000000,-0.174368,-0.503460
wind_speed,0.190590,0.105082,0.067732,0.118649,0.216336,-0.174368,1.000000,-0.012113
relative_humidity,-0.431754,-0.254196,0.006976,-0.552613,-0.496328,-0.503460,-0.012113,1.000000


### Adding lag to the average dataframe for further correlation help

In [ ]:
def lagged_col_rev(df, col, lag, fill_value=0):
    df[col+'_lag_-'+str(lag)] = df[col].shift(periods=-lag)
    return df
lags = 7 

Covid_Data_avg_lag = Covid_Data_avg.copy()
for lag in range(1,lags+1):
    Covid_avg_PCR_lag = lagged_col_rev(Covid_Data_avg, 'PCR_Postive_Count', lag, fill_value=0)
Covid_avg_PCR_lag    

    


,Date,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,Day_of_week,PCR_Postive_Count_lag_-1,PCR_Postive_Count_lag_-2,PCR_Postive_Count_lag_-3,PCR_Postive_Count_lag_-4,PCR_Postive_Count_lag_-5,PCR_Postive_Count_lag_-6,PCR_Postive_Count_lag_-7
0,2021-08-02,122,3,0,138639,1560,15.408933,10.650933,83.221333,Monday,142.0,170.0,129.0,156.0,51.0,47.0,115.0
1,2021-08-03,142,8,0,138781,1646,14.946933,10.774667,87.574667,Tuesday,170.0,129.0,156.0,51.0,47.0,115.0,161.0
2,2021-08-04,170,6,1,138951,1758,16.023067,7.270933,81.329333,Wednesday,129.0,156.0,51.0,47.0,115.0,161.0,234.0
3,2021-08-05,129,1,0,139080,1830,17.316933,9.500400,80.693333,Thursday,156.0,51.0,47.0,115.0,161.0,234.0,247.0
4,2021-08-06,156,10,3,139236,1901,16.960667,16.658667,80.053333,Friday,51.0,47.0,115.0,161.0,234.0,247.0,228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,2022-07-25,6598,63,14,1184097,84698,24.600400,15.073067,51.744000,Monday,5792.0,4841.0,3913.0,3133.0,NaN,NaN,NaN
329,2022-07-26,5792,137,6,1189889,89989,20.124400,12.751867,58.258667,Tuesday,4841.0,3913.0,3133.0,NaN,NaN,NaN,NaN
330,2022-07-27,4841,104,3,1194730,94494,18.504533,12.537733,54.682667,Wednesday,3913.0,3133.0,NaN,NaN,NaN,NaN,NaN
331,2022-07-28,3913,76,7,1198643,92589,20.134000,12.251200,49.390667,Thursday,3133.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for lag in range(1,lags+1):
    Covid_avg_CrIn_lag = lagged_col_rev(Covid_Data_avg, 'Current_Infected', lag, fill_value=0)
Covid_avg_CrIn_lag

,Date,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,Day_of_week,...,PCR_Postive_Count_lag_-5,PCR_Postive_Count_lag_-6,PCR_Postive_Count_lag_-7,Current_Infected_lag_-1,Current_Infected_lag_-2,Current_Infected_lag_-3,Current_Infected_lag_-4,Current_Infected_lag_-5,Current_Infected_lag_-6,Current_Infected_lag_-7
0,2021-08-02,122,3,0,138639,1560,15.408933,10.650933,83.221333,Monday,...,51.0,47.0,115.0,1646.0,1758.0,1830.0,1901.0,1875.0,1834.0,1870.0
1,2021-08-03,142,8,0,138781,1646,14.946933,10.774667,87.574667,Tuesday,...,47.0,115.0,161.0,1758.0,1830.0,1901.0,1875.0,1834.0,1870.0,1959.0
2,2021-08-04,170,6,1,138951,1758,16.023067,7.270933,81.329333,Wednesday,...,115.0,161.0,234.0,1830.0,1901.0,1875.0,1834.0,1870.0,1959.0,2138.0
3,2021-08-05,129,1,0,139080,1830,17.316933,9.500400,80.693333,Thursday,...,161.0,234.0,247.0,1901.0,1875.0,1834.0,1870.0,1959.0,2138.0,2291.0
4,2021-08-06,156,10,3,139236,1901,16.960667,16.658667,80.053333,Friday,...,234.0,247.0,228.0,1875.0,1834.0,1870.0,1959.0,2138.0,2291.0,2440.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,2022-07-25,6598,63,14,1184097,84698,24.600400,15.073067,51.744000,Monday,...,NaN,NaN,NaN,89989.0,94494.0,92589.0,90382.0,NaN,NaN,NaN
329,2022-07-26,5792,137,6,1189889,89989,20.124400,12.751867,58.258667,Tuesday,...,NaN,NaN,NaN,94494.0,92589.0,90382.0,NaN,NaN,NaN,NaN
330,2022-07-27,4841,104,3,1194730,94494,18.504533,12.537733,54.682667,Wednesday,...,NaN,NaN,NaN,92589.0,90382.0,NaN,NaN,NaN,NaN,NaN
331,2022-07-28,3913,76,7,1198643,92589,20.134000,12.251200,49.390667,Thursday,...,NaN,NaN,NaN,90382.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
!pwd

/content


In [ ]:
corr_temp = Covid_avg_PCR_lag.corr(method='spearman')
#corr_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/PCR_Lag_Spear_Corr.csv')
corr_temp

,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,PCR_Postive_Count_lag_-1,PCR_Postive_Count_lag_-2,...,PCR_Postive_Count_lag_-5,PCR_Postive_Count_lag_-6,PCR_Postive_Count_lag_-7,Current_Infected_lag_-1,Current_Infected_lag_-2,Current_Infected_lag_-3,Current_Infected_lag_-4,Current_Infected_lag_-5,Current_Infected_lag_-6,Current_Infected_lag_-7
PCR_Postive_Count,1.000000,0.821788,0.563022,0.784037,0.859562,-0.148613,0.190590,-0.431754,0.889676,0.766969,...,0.761237,0.867639,0.936996,0.870936,0.870477,0.864692,0.861962,0.865465,0.877177,0.891136
PCR_Positive_Hospitalised,0.821788,1.000000,0.738336,0.569711,0.601362,-0.079667,0.105082,-0.254196,0.634822,0.433994,...,0.431427,0.630962,0.738072,0.608622,0.597229,0.579298,0.564935,0.561844,0.573360,0.589583
Deceased,0.563022,0.738336,1.000000,0.325752,0.385715,-0.243298,0.067732,0.006976,0.435891,0.248485,...,0.156711,0.339041,0.508644,0.397692,0.389114,0.371303,0.354866,0.345572,0.351426,0.366196
Total_Confirmed,0.784037,0.569711,0.325752,1.000000,0.818746,0.090358,0.118649,-0.552613,0.782125,0.780258,...,0.774525,0.772451,0.770350,0.817103,0.815440,0.813757,0.812054,0.810332,0.808587,0.806820
Current_Infected,0.859562,0.601362,0.385715,0.818746,1.000000,-0.206005,0.216336,-0.496328,0.839541,0.825919,...,0.820642,0.817867,0.805274,0.997972,0.994852,0.991792,0.989360,0.987092,0.983881,0.979266
temperature,-0.148613,-0.079667,-0.243298,0.090358,-0.206005,1.000000,-0.174368,-0.503460,-0.157304,-0.182010,...,-0.215331,-0.212508,-0.206933,-0.215612,-0.226282,-0.238892,-0.251718,-0.263332,-0.273650,-0.283382
wind_speed,0.190590,0.105082,0.067732,0.118649,0.216336,-0.174368,1.000000,-0.012113,0.147815,0.145398,...,0.224722,0.198118,0.188201,0.219622,0.215574,0.214753,0.222385,0.226544,0.223807,0.223351
relative_humidity,-0.431754,-0.254196,0.006976,-0.552613,-0.496328,-0.503460,-0.012113,1.000000,-0.435322,-0.437853,...,-0.407519,-0.373872,-0.380395,-0.489828,-0.484248,-0.480246,-0.475326,-0.469526,-0.459001,-0.451159
PCR_Postive_Count_lag_-1,0.889676,0.634822,0.435891,0.782125,0.839541,-0.157304,0.147815,-0.435322,1.000000,0.888765,...,0.714104,0.759148,0.866498,0.858318,0.869790,0.869324,0.863485,0.860734,0.864266,0.876076
PCR_Postive_Count_lag_-2,0.766969,0.433994,0.248485,0.780258,0.825919,-0.182010,0.145398,-0.437853,0.888765,1.000000,...,0.718975,0.711680,0.757101,0.838187,0.857127,0.868698,0.868229,0.862326,0.859549,0.863110


In [ ]:
corr_temp1 = Covid_avg_CrIn_lag.corr(method='spearman')
#corr_temp1.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/CrIn_Lag_Spear_Corr.csv')
corr_temp1

,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,PCR_Postive_Count_lag_-1,PCR_Postive_Count_lag_-2,...,PCR_Postive_Count_lag_-5,PCR_Postive_Count_lag_-6,PCR_Postive_Count_lag_-7,Current_Infected_lag_-1,Current_Infected_lag_-2,Current_Infected_lag_-3,Current_Infected_lag_-4,Current_Infected_lag_-5,Current_Infected_lag_-6,Current_Infected_lag_-7
PCR_Postive_Count,1.000000,0.821788,0.563022,0.784037,0.859562,-0.148613,0.190590,-0.431754,0.889676,0.766969,...,0.761237,0.867639,0.936996,0.870936,0.870477,0.864692,0.861962,0.865465,0.877177,0.891136
PCR_Positive_Hospitalised,0.821788,1.000000,0.738336,0.569711,0.601362,-0.079667,0.105082,-0.254196,0.634822,0.433994,...,0.431427,0.630962,0.738072,0.608622,0.597229,0.579298,0.564935,0.561844,0.573360,0.589583
Deceased,0.563022,0.738336,1.000000,0.325752,0.385715,-0.243298,0.067732,0.006976,0.435891,0.248485,...,0.156711,0.339041,0.508644,0.397692,0.389114,0.371303,0.354866,0.345572,0.351426,0.366196
Total_Confirmed,0.784037,0.569711,0.325752,1.000000,0.818746,0.090358,0.118649,-0.552613,0.782125,0.780258,...,0.774525,0.772451,0.770350,0.817103,0.815440,0.813757,0.812054,0.810332,0.808587,0.806820
Current_Infected,0.859562,0.601362,0.385715,0.818746,1.000000,-0.206005,0.216336,-0.496328,0.839541,0.825919,...,0.820642,0.817867,0.805274,0.997972,0.994852,0.991792,0.989360,0.987092,0.983881,0.979266
temperature,-0.148613,-0.079667,-0.243298,0.090358,-0.206005,1.000000,-0.174368,-0.503460,-0.157304,-0.182010,...,-0.215331,-0.212508,-0.206933,-0.215612,-0.226282,-0.238892,-0.251718,-0.263332,-0.273650,-0.283382
wind_speed,0.190590,0.105082,0.067732,0.118649,0.216336,-0.174368,1.000000,-0.012113,0.147815,0.145398,...,0.224722,0.198118,0.188201,0.219622,0.215574,0.214753,0.222385,0.226544,0.223807,0.223351
relative_humidity,-0.431754,-0.254196,0.006976,-0.552613,-0.496328,-0.503460,-0.012113,1.000000,-0.435322,-0.437853,...,-0.407519,-0.373872,-0.380395,-0.489828,-0.484248,-0.480246,-0.475326,-0.469526,-0.459001,-0.451159
PCR_Postive_Count_lag_-1,0.889676,0.634822,0.435891,0.782125,0.839541,-0.157304,0.147815,-0.435322,1.000000,0.888765,...,0.714104,0.759148,0.866498,0.858318,0.869790,0.869324,0.863485,0.860734,0.864266,0.876076
PCR_Postive_Count_lag_-2,0.766969,0.433994,0.248485,0.780258,0.825919,-0.182010,0.145398,-0.437853,0.888765,1.000000,...,0.718975,0.711680,0.757101,0.838187,0.857127,0.868698,0.868229,0.862326,0.859549,0.863110


In [ ]:
corr_temp = Covid_avg_PCR_lag.corr(method='kendall')
#corr_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/PCR_Lag_kendall_Corr.csv')
corr_temp

,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,PCR_Postive_Count_lag_-1,PCR_Postive_Count_lag_-2,...,PCR_Postive_Count_lag_-5,PCR_Postive_Count_lag_-6,PCR_Postive_Count_lag_-7,Current_Infected_lag_-1,Current_Infected_lag_-2,Current_Infected_lag_-3,Current_Infected_lag_-4,Current_Infected_lag_-5,Current_Infected_lag_-6,Current_Infected_lag_-7
PCR_Postive_Count,1.000000,0.629196,0.401049,0.575861,0.683771,-0.103694,0.126492,-0.281842,0.737142,0.568608,...,0.564708,0.699529,0.799196,0.701155,0.702786,0.697995,0.696264,0.703619,0.722066,0.742936
PCR_Positive_Hospitalised,0.629196,1.000000,0.550917,0.400143,0.458070,-0.050982,0.070836,-0.169890,0.472756,0.307382,...,0.307478,0.453444,0.546091,0.463016,0.454592,0.442942,0.430973,0.427849,0.434571,0.443868
Deceased,0.401049,0.550917,1.000000,0.231779,0.282008,-0.170203,0.046295,0.002912,0.306283,0.175097,...,0.101969,0.231931,0.354123,0.289873,0.283902,0.270571,0.259201,0.251938,0.253526,0.263616
Total_Confirmed,0.575861,0.400143,0.231779,1.000000,0.633561,0.053909,0.073049,-0.354246,0.573553,0.571384,...,0.564888,0.562255,0.559561,0.631347,0.629113,0.626858,0.624583,0.622399,0.620157,0.617820
Current_Infected,0.683771,0.458070,0.282008,0.633561,1.000000,-0.136619,0.142299,-0.310395,0.656594,0.639801,...,0.627141,0.623052,0.607327,0.965093,0.940932,0.923441,0.910631,0.901581,0.891297,0.876848
temperature,-0.103694,-0.050982,-0.170203,0.053909,-0.136619,1.000000,-0.116357,-0.349361,-0.108034,-0.120701,...,-0.145787,-0.143848,-0.141939,-0.141302,-0.147377,-0.157705,-0.166432,-0.173976,-0.180691,-0.186937
wind_speed,0.126492,0.070836,0.046295,0.073049,0.142299,-0.116357,1.000000,-0.008937,0.096420,0.095978,...,0.148398,0.130975,0.126797,0.144942,0.141078,0.140204,0.145971,0.148393,0.147070,0.147522
relative_humidity,-0.281842,-0.169890,0.002912,-0.354246,-0.310395,-0.349361,-0.008937,1.000000,-0.280779,-0.278962,...,-0.260111,-0.241497,-0.243437,-0.308594,-0.305356,-0.302938,-0.300801,-0.298575,-0.291608,-0.288759
PCR_Postive_Count_lag_-1,0.737142,0.472756,0.306283,0.573553,0.656594,-0.108034,0.096420,-0.280779,1.000000,0.735926,...,0.511397,0.562337,0.697944,0.682115,0.699600,0.701237,0.696412,0.694779,0.702138,0.720620
PCR_Postive_Count_lag_-2,0.568608,0.307382,0.175097,0.571384,0.639801,-0.120701,0.095978,-0.278962,0.735926,1.000000,...,0.515179,0.508863,0.560072,0.654928,0.680603,0.698191,0.699835,0.695015,0.693331,0.700733


In [ ]:
corr_temp1 = Covid_avg_CrIn_lag.corr(method='kendall')
#corr_temp1.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/CrIn_Lag_kendall_Corr.csv')
corr_temp1

,PCR_Postive_Count,PCR_Positive_Hospitalised,Deceased,Total_Confirmed,Current_Infected,temperature,wind_speed,relative_humidity,PCR_Postive_Count_lag_-1,PCR_Postive_Count_lag_-2,...,PCR_Postive_Count_lag_-5,PCR_Postive_Count_lag_-6,PCR_Postive_Count_lag_-7,Current_Infected_lag_-1,Current_Infected_lag_-2,Current_Infected_lag_-3,Current_Infected_lag_-4,Current_Infected_lag_-5,Current_Infected_lag_-6,Current_Infected_lag_-7
PCR_Postive_Count,1.000000,0.629196,0.401049,0.575861,0.683771,-0.103694,0.126492,-0.281842,0.737142,0.568608,...,0.564708,0.699529,0.799196,0.701155,0.702786,0.697995,0.696264,0.703619,0.722066,0.742936
PCR_Positive_Hospitalised,0.629196,1.000000,0.550917,0.400143,0.458070,-0.050982,0.070836,-0.169890,0.472756,0.307382,...,0.307478,0.453444,0.546091,0.463016,0.454592,0.442942,0.430973,0.427849,0.434571,0.443868
Deceased,0.401049,0.550917,1.000000,0.231779,0.282008,-0.170203,0.046295,0.002912,0.306283,0.175097,...,0.101969,0.231931,0.354123,0.289873,0.283902,0.270571,0.259201,0.251938,0.253526,0.263616
Total_Confirmed,0.575861,0.400143,0.231779,1.000000,0.633561,0.053909,0.073049,-0.354246,0.573553,0.571384,...,0.564888,0.562255,0.559561,0.631347,0.629113,0.626858,0.624583,0.622399,0.620157,0.617820
Current_Infected,0.683771,0.458070,0.282008,0.633561,1.000000,-0.136619,0.142299,-0.310395,0.656594,0.639801,...,0.627141,0.623052,0.607327,0.965093,0.940932,0.923441,0.910631,0.901581,0.891297,0.876848
temperature,-0.103694,-0.050982,-0.170203,0.053909,-0.136619,1.000000,-0.116357,-0.349361,-0.108034,-0.120701,...,-0.145787,-0.143848,-0.141939,-0.141302,-0.147377,-0.157705,-0.166432,-0.173976,-0.180691,-0.186937
wind_speed,0.126492,0.070836,0.046295,0.073049,0.142299,-0.116357,1.000000,-0.008937,0.096420,0.095978,...,0.148398,0.130975,0.126797,0.144942,0.141078,0.140204,0.145971,0.148393,0.147070,0.147522
relative_humidity,-0.281842,-0.169890,0.002912,-0.354246,-0.310395,-0.349361,-0.008937,1.000000,-0.280779,-0.278962,...,-0.260111,-0.241497,-0.243437,-0.308594,-0.305356,-0.302938,-0.300801,-0.298575,-0.291608,-0.288759
PCR_Postive_Count_lag_-1,0.737142,0.472756,0.306283,0.573553,0.656594,-0.108034,0.096420,-0.280779,1.000000,0.735926,...,0.511397,0.562337,0.697944,0.682115,0.699600,0.701237,0.696412,0.694779,0.702138,0.720620
PCR_Postive_Count_lag_-2,0.568608,0.307382,0.175097,0.571384,0.639801,-0.120701,0.095978,-0.278962,0.735926,1.000000,...,0.515179,0.508863,0.560072,0.654928,0.680603,0.698191,0.699835,0.695015,0.693331,0.700733


### Correlation between Temperature and Relative Humidity

In [ ]:
data = Covid_Data1

corr_kendall= s.kendalltau(data['temperature'],data['relative_humidity'])
print('Kendall-Tau Correlation between Temperature and Relative Humidity is',round(corr_kendall[0],4), 'for p-value', corr_kendall[1], 'and hence significance as',corr_kendall[1] <= 0.05)

corr_spear= spearmanr(data['temperature'],data['relative_humidity'])
print('Spearmann Correlation between Temperature and Relative Humidity is',round(corr_spear[0],4), 'for p-value', corr_spear[1], 'and hence significance as',corr_spear[1] <= 0.05) 

data1 = Covid_Data_avg

corr_kendall1= s.kendalltau(data1['temperature'],data1['relative_humidity'])
print('Kendall-Tau Correlation between Temperature and Relative Humidity for Average State(RLP) data is',round(corr_kendall1[0],4), 'for p-value', corr_kendall1[1], 'and hence significance as',corr_kendall1[1] <= 0.05)

corr_spear1= spearmanr(data1['temperature'],data1['relative_humidity'])
print('Spearmann Correlation between Temperature and Relative Humidity for Average State(RLP) data is',round(corr_spear1[0],4), 'for p-value', corr_spear1[1], 'and hence significance as',corr_spear1[1] <= 0.05) 

Kendall-Tau Correlation between Temperature and Relative Humidity is -0.3398 for p-value 0.0 and hence significance as True
Spearmann Correlation between Temperature and Relative Humidity is -0.4889 for p-value 0.0 and hence significance as True
Kendall-Tau Correlation between Temperature and Relative Humidity for Average State(RLP) data is -0.3494 for p-value 1.8549202795204935e-21 and hence significance as True
Spearmann Correlation between Temperature and Relative Humidity for Average State(RLP) data is -0.5035 for p-value 8.415323123118558e-23 and hence significance as True


In [ ]:
data = Covid_Data_avg.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temperature", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,35])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,110])

fig.update_layout(plot_bgcolor="white")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    bordercolor="Black",
    borderwidth=1
))


fig.show()

#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/Avg_Temp_Hum.png")

#### Showing graph with various seasons

In [ ]:
data = Covid_Data_avg.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temperature", mode='lines',line_color = 'royalblue'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="Humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,35])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered', range = [30,110])

# Add shape regions
fig.add_vrect(
    x0="2021-08-01", x1="2021-08-31",
    fillcolor="lightblue", opacity=0.5,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2021-09-01", x1="2021-11-30",
    fillcolor="lightblue", opacity=0.3,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2021-12-01", x1="2022-02-28",
    fillcolor="aliceblue", opacity=0.8,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2022-03-01", x1="2022-05-31",
    fillcolor="aliceblue", opacity=0.2,
    layer="below", line_width=0,
)

fig.add_vrect(
    x0="2022-06-01", x1="2022-07-31",
    fillcolor="lightblue", opacity=0.5,
    layer="below", line_width=0,
)

fig.update_layout(plot_bgcolor="white")

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01,
    bordercolor="Black",
    borderwidth=1
))

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/Avg_Temp_Hum_seasons.png")

### Correlation


In [ ]:
"""
Correlation:
1. Positive: when one variable moves higher or lower, the other variable moves in the same direction with the same magnitude.
2. Negative: if one variable increases, the other variable decreases with the same magnitude (and vice versa).
"""

'\nCorrelation:\n1. Positive: when one variable moves higher or lower, the other variable moves in the same direction with the same magnitude.\n2. Negative: if one variable increases, the other variable decreases with the same magnitude (and vice versa).\n'

In [ ]:
#correlation matrix --- Kendall

def find_cor_sig(data,final_list,season_name,covid_param,weather_param):
   # covid_param = ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased', 'Current_Infected', 'Total_Confirmed']
   # weather_param = ['temperature','wind_speed','relative_humidity']
    return_list = []
    for i in range(0,len(covid_param)):
        for j in range(0,len(weather_param)):
            corr_kendall= kendalltau(data[covid_param[i]],data[weather_param[j]])
            corr = round(corr_kendall[0],4)
            sig = corr_kendall[1] <= 0.05
            return_list.append(('Kendall-Tau',season_name,covid_param[i],weather_param[j],corr,corr_kendall[1],sig))
    for i in range(0,len(covid_param)):
        for j in range(0,len(weather_param)):
            corr_spear= spearmanr(data[covid_param[i]],data[weather_param[j]])
            corr = round(corr_spear[0],4)
            sig = corr_spear[1] <= 0.05
            return_list.append(('Spearmann',season_name,covid_param[i],weather_param[j],corr,corr_spear[1],sig))        
        
    #corr_kendall1= data.corr(method='kendall')
    for val in range(0,len(return_list)):
        final_list.append(return_list[val])
    return final_list

In [ ]:
data = Covid_Data1.copy()
final_list = []
covid_param = ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased', 'Current_Infected', 'Total_Confirmed']
weather_param = ['temperature','wind_speed','relative_humidity']
final_list = find_cor_sig(data,final_list,'Total',covid_param, weather_param)

corr_data = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
#corr_data.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/Correlation.csv')
corr_data_final = corr_data
corr_data

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Total,PCR_Postive_Count,temperature,-0.0470,2.643696e-12,True
1,Kendall-Tau,Total,PCR_Postive_Count,wind_speed,0.0450,2.195353e-11,True
2,Kendall-Tau,Total,PCR_Postive_Count,relative_humidity,-0.2137,9.521208e-222,True
3,Kendall-Tau,Total,PCR_Positive_Hospitalised,temperature,-0.0311,3.133476e-05,True
4,Kendall-Tau,Total,PCR_Positive_Hospitalised,wind_speed,0.0156,3.597302e-02,True
5,Kendall-Tau,Total,PCR_Positive_Hospitalised,relative_humidity,-0.0613,2.074113e-16,True
6,Kendall-Tau,Total,Deceased,temperature,-0.0729,1.499910e-19,True
7,Kendall-Tau,Total,Deceased,wind_speed,0.0301,1.917699e-04,True
8,Kendall-Tau,Total,Deceased,relative_humidity,0.0015,8.503016e-01,False
9,Kendall-Tau,Total,Current_Infected,temperature,-0.1209,2.351090e-73,True


In [ ]:
corr_data_pos = corr_data.loc[(corr_data["Significance"]==True)]
#corr_data_pos

#Overall temperature has a very negligible correlation with daily count

### Doing Correlation with Weather

In [ ]:
corr_data_pos.loc[(corr_data_pos["Weather_Column"]=='temperature')]

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Total,PCR_Postive_Count,temperature,-0.0470,2.643696e-12,True
3,Kendall-Tau,Total,PCR_Positive_Hospitalised,temperature,-0.0311,3.133476e-05,True
6,Kendall-Tau,Total,Deceased,temperature,-0.0729,1.499910e-19,True
9,Kendall-Tau,Total,Current_Infected,temperature,-0.1209,2.351090e-73,True
12,Kendall-Tau,Total,Total_Confirmed,temperature,0.1174,2.538583e-69,True
15,Spearmann,Total,PCR_Postive_Count,temperature,-0.0732,2.346434e-13,True
18,Spearmann,Total,PCR_Positive_Hospitalised,temperature,-0.0420,2.718374e-05,True
21,Spearmann,Total,Deceased,temperature,-0.0903,1.470404e-19,True
24,Spearmann,Total,Current_Infected,temperature,-0.1844,4.354628e-77,True
27,Spearmann,Total,Total_Confirmed,temperature,0.1759,2.907073e-70,True


In [ ]:
data = Covid_Data_avg.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green', range = [0,14500])

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/PCR_Temp.png")

In [ ]:
data = Covid_Data_avg.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="Temp", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Temperature [°C]", title_font_color= 'royalblue',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='royalblue',
                 showticklabels = True, ticks="inside", tickfont_color = 'royalblue', range = [-5,30])

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/CrIn_Temp.png")

In [ ]:
#correlation for lagged data
data = Covid_avg_PCR_lag.copy()
final_list = []
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1',
       'PCR_Postive_Count_lag_-2', 'PCR_Postive_Count_lag_-3',
       'PCR_Postive_Count_lag_-4', 'PCR_Postive_Count_lag_-5',
       'PCR_Postive_Count_lag_-6', 'PCR_Postive_Count_lag_-7']
weather_param = ['temperature']
final_list = find_cor_sig(data,final_list,'Total', covid_param, weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
#corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/PCR_Lag_Temp_Correlation.csv')
corr_data_temp
#All significance false values


,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Total,PCR_Postive_Count,temperature,-0.1037,0.004757,True
1,Kendall-Tau,Total,PCR_Postive_Count_lag_-1,temperature,-0.1017,0.005609,True
2,Kendall-Tau,Total,PCR_Postive_Count_lag_-2,temperature,-0.1084,0.003171,True
3,Kendall-Tau,Total,PCR_Postive_Count_lag_-3,temperature,-0.1224,0.000860,True
4,Kendall-Tau,Total,PCR_Postive_Count_lag_-4,temperature,-0.1211,0.000978,True
5,Kendall-Tau,Total,PCR_Postive_Count_lag_-5,temperature,-0.1149,0.001757,True
6,Kendall-Tau,Total,PCR_Postive_Count_lag_-6,temperature,-0.1065,0.003745,True
7,Kendall-Tau,Total,PCR_Postive_Count_lag_-7,temperature,-0.0985,0.007315,True
8,Spearmann,Total,PCR_Postive_Count,temperature,-0.1486,0.006591,True
9,Spearmann,Total,PCR_Postive_Count_lag_-1,temperature,-0.1472,0.007117,True


In [ ]:
#correlation for lagged data
data = Covid_avg_CrIn_lag.copy()
data = data.dropna()
final_list = []
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2',
       'Current_Infected_lag_-3', 'Current_Infected_lag_-4',
       'Current_Infected_lag_-5', 'Current_Infected_lag_-6',
       'Current_Infected_lag_-7']
weather_param = ['temperature']
final_list = find_cor_sig(data,final_list,'Total',covid_param, weather_param)
corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
#corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/CrIn_Lag_Temp_Correlation.csv')
corr_data_temp
#All significance false values

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Total,Current_Infected,temperature,-0.1577,2.150759e-05,True
1,Kendall-Tau,Total,Current_Infected_lag_-1,temperature,-0.1596,1.719807e-05,True
2,Kendall-Tau,Total,Current_Infected_lag_-2,temperature,-0.1630,1.133490e-05,True
3,Kendall-Tau,Total,Current_Infected_lag_-3,temperature,-0.1709,4.132671e-06,True
4,Kendall-Tau,Total,Current_Infected_lag_-4,temperature,-0.1766,1.953585e-06,True
5,Kendall-Tau,Total,Current_Infected_lag_-5,temperature,-0.1808,1.115754e-06,True
6,Kendall-Tau,Total,Current_Infected_lag_-6,temperature,-0.1840,7.215532e-07,True
7,Kendall-Tau,Total,Current_Infected_lag_-7,temperature,-0.1869,4.758171e-07,True
8,Spearmann,Total,Current_Infected,temperature,-0.2383,1.369396e-05,True
9,Spearmann,Total,Current_Infected_lag_-1,temperature,-0.2433,8.896600e-06,True


In [ ]:
data = Covid_Data_avg.copy()
#print(data)
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="PCR_Postive_Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count_lag_-3'],name="PCR_Postive_Count lag 3", line=dict(color='green',
                              dash='dot')),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['temperature'],name="temperature", mode='lines',line_color = 'royalblue'),
    secondary_y=True)

fig.show()

### Doing Correlation with Relative Humidity

In [ ]:
corr_data_pos.loc[(corr_data_pos["Weather_Column"]=='relative_humidity')]

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
2,Kendall-Tau,Total,PCR_Postive_Count,relative_humidity,-0.2137,9.521208e-222,True
5,Kendall-Tau,Total,PCR_Positive_Hospitalised,relative_humidity,-0.0613,2.074113e-16,True
11,Kendall-Tau,Total,Current_Infected,relative_humidity,-0.2749,0.000000e+00,True
14,Kendall-Tau,Total,Total_Confirmed,relative_humidity,-0.3562,0.000000e+00,True
17,Spearmann,Total,PCR_Postive_Count,relative_humidity,-0.3274,3.538920e-248,True
20,Spearmann,Total,PCR_Positive_Hospitalised,relative_humidity,-0.0831,8.748424e-17,True
26,Spearmann,Total,Current_Infected,relative_humidity,-0.4341,0.000000e+00,True
29,Spearmann,Total,Total_Confirmed,relative_humidity,-0.5391,0.000000e+00,True


In [ ]:
data = Covid_Data_avg.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['PCR_Postive_Count'],name="Daily Positive Count", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="relative_humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Daily Positive Count", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green', range = [0,14500])

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/PCR_Hum.png")

In [ ]:
data = Covid_Data_avg.copy()
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Current_Infected'],name="Current Infected", mode='lines',line_color = 'green'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=data['Date'], y=data['relative_humidity'],name="relative_humidity", mode='lines',line_color = 'orangered'),
    secondary_y=True)

# Set x-axis title
fig.update_xaxes(title_text="Date", showgrid=True, gridwidth=0.5, gridcolor='lightgray', 
                 showline=True, linewidth=1, linecolor='black', mirror=True, 
                 showticklabels = True, ticks="inside", dtick="M1", ticklabelmode="period")

# Set y-axes titles
fig.update_yaxes(title_text="Current Infected", title_font_color= 'green',  secondary_y=False, 
                 showgrid=True, gridwidth=0.5, gridcolor='lightskyblue',
                 zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='green',
                 showticklabels = True, ticks="inside", tickfont_color = 'green')

fig.update_yaxes(title_text="Average Humidity [%]", title_font_color= 'orangered',  secondary_y=True, 
                # showgrid=True, gridwidth=0.5, gridcolor='lightskyblue', 
                # zeroline = True, zerolinewidth = 0.5, zerolinecolor = 'lightskyblue',
                 showline=True, linewidth=1.5, linecolor='orangered',
                 showticklabels = True, ticks="inside", tickfont_color = 'orangered')

fig.update_layout(plot_bgcolor="white")

fig.show()
#fig.write_image("gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/CrIn_Hum.png")

In [ ]:
#correlation for lagged data
data = Covid_avg_PCR_lag.copy()
final_list = []
covid_param = ['PCR_Postive_Count','PCR_Postive_Count_lag_-1',
       'PCR_Postive_Count_lag_-2', 'PCR_Postive_Count_lag_-3',
       'PCR_Postive_Count_lag_-4', 'PCR_Postive_Count_lag_-5',
       'PCR_Postive_Count_lag_-6', 'PCR_Postive_Count_lag_-7']
weather_param = ['relative_humidity']
final_list = find_cor_sig(data,final_list,'Total',covid_param, weather_param)

corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
#corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/PCR_Lag_Hum_Correlation.csv')
corr_data_temp
#All significance false values

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Total,PCR_Postive_Count,relative_humidity,-0.2818,1.679379e-14,True
1,Kendall-Tau,Total,PCR_Postive_Count_lag_-1,relative_humidity,-0.2841,1.032987e-14,True
2,Kendall-Tau,Total,PCR_Postive_Count_lag_-2,relative_humidity,-0.2861,6.711268e-15,True
3,Kendall-Tau,Total,PCR_Postive_Count_lag_-3,relative_humidity,-0.2875,4.947981e-15,True
4,Kendall-Tau,Total,PCR_Postive_Count_lag_-4,relative_humidity,-0.2868,5.854452e-15,True
5,Kendall-Tau,Total,PCR_Postive_Count_lag_-5,relative_humidity,-0.2773,4.388554e-14,True
6,Kendall-Tau,Total,PCR_Postive_Count_lag_-6,relative_humidity,-0.2612,1.156035e-12,True
7,Kendall-Tau,Total,PCR_Postive_Count_lag_-7,relative_humidity,-0.2640,6.653053e-13,True
8,Spearmann,Total,PCR_Postive_Count,relative_humidity,-0.4318,1.489068e-16,True
9,Spearmann,Total,PCR_Postive_Count_lag_-1,relative_humidity,-0.4397,3.529343e-17,True


In [ ]:
#correlation for lagged data
data = Covid_avg_CrIn_lag.copy()
data = data.dropna()
final_list = []
covid_param = ['Current_Infected','Current_Infected_lag_-1', 'Current_Infected_lag_-2',
       'Current_Infected_lag_-3', 'Current_Infected_lag_-4',
       'Current_Infected_lag_-5', 'Current_Infected_lag_-6',
       'Current_Infected_lag_-7']
weather_param = ['relative_humidity']
final_list = find_cor_sig(data,final_list,'Total',covid_param, weather_param)
corr_data_temp = pd.DataFrame (final_list, columns = ['CorrelationMethod','Season','Covid_Column','Weather_Column','Correlation','Significance_value','Significance'])
#corr_data_temp.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/CrIn_Lag_Hum_Correlation.csv')
corr_data_temp

,CorrelationMethod,Season,Covid_Column,Weather_Column,Correlation,Significance_value,Significance
0,Kendall-Tau,Total,Current_Infected,relative_humidity,-0.3036,2.851401e-16,True
1,Kendall-Tau,Total,Current_Infected_lag_-1,relative_humidity,-0.3025,3.702277e-16,True
2,Kendall-Tau,Total,Current_Infected_lag_-2,relative_humidity,-0.2997,6.764439e-16,True
3,Kendall-Tau,Total,Current_Infected_lag_-3,relative_humidity,-0.2987,8.468392e-16,True
4,Kendall-Tau,Total,Current_Infected_lag_-4,relative_humidity,-0.2979,1.024851e-15,True
5,Kendall-Tau,Total,Current_Infected_lag_-5,relative_humidity,-0.2966,1.357438e-15,True
6,Kendall-Tau,Total,Current_Infected_lag_-6,relative_humidity,-0.2909,4.624958e-15,True
7,Kendall-Tau,Total,Current_Infected_lag_-7,relative_humidity,-0.2888,7.324941e-15,True
8,Spearmann,Total,Current_Infected,relative_humidity,-0.4842,1.442079e-20,True
9,Spearmann,Total,Current_Infected_lag_-1,relative_humidity,-0.4792,4.019829e-20,True


## Multiple Correlation
Multiple correlation of complete dataset is left!!!

In [ ]:
#combined multiple correlations
def find_mul_cor(AB,CB,AC, n):
    value =((AB**2) + (CB**2) - (2*AB*CB*AC))/(1 -(AC**2))

    r = math.sqrt(value)

    r_squared = r**2

    correlation_C_WRT_AB = (1- (((1-r_squared)*(n-1))/(n-2-1)))

    return round(correlation_C_WRT_AB,4)

https://www.real-statistics.com/correlation/multiple-correlation/

Understanding multiple coreelation R, Adj R sqaure and R square

the coefficient of multiple correlation is a measure of how well a given variable can be predicted using a linear function of a set of other variables.

In [ ]:
mul_corr_list = []
#-----------fetching rows only with significance value=TRUE--------------
mul_corr_data = corr_data_final.loc[(corr_data_final["Significance"]==True)]

data = Covid_Data_avg.copy()

#-----correlation between temperature and relative humidity--------------
corr_kendall= kendalltau(data['temperature'],data['relative_humidity'])
corr_TR_ken = round(corr_kendall[0],4)

corr_spear= spearmanr(data['temperature'],data['relative_humidity'])
corr_TR_spe = round(corr_spear[0],4)

n=333
#mul_corr_data.tail(10)



In [ ]:
covid_param =  ['PCR_Postive_Count','PCR_Positive_Hospitalised','Deceased', 'Current_Infected', 'Total_Confirmed']
weather_param = ['temperature','relative_humidity']
correlation = ['Kendall-Tau','Spearmann']
corr_mul_list=[]

for i in correlation:
    for j in covid_param:
        result_list=[]
        for k in weather_param:
            #print(i,j,k,l)
            temp = mul_corr_data.loc[(mul_corr_data["Covid_Column"]== j) & (mul_corr_data['CorrelationMethod']==i) & (mul_corr_data['Weather_Column']==k)]
            #print(temp['Correlation'].array)
            new_data = []
            new_data = temp['Correlation'].array
            if (len(new_data)>0):
                result_list.append(new_data[0])
                #corr = temp.iloc[0].Correlation
                #print(corr)
                
            #print(i,j,k,result_list)
        if(len(result_list)==2):
            if(i=='Kendall-Tau'):
                result = find_mul_cor(corr_TR_ken,result_list[0],result_list[1], n)
                corr_mul_list.append(('Kendall-Tau','Total',j,result_list[0],result_list[1],result))
            else:
                result = find_mul_cor(corr_TR_spe,result_list[0],result_list[1], n)
                corr_mul_list.append(('Spearmann','Total',j,result_list[0],result_list[1],result))
                    

corr_mul_TH = pd.DataFrame (corr_mul_list, columns = ['CorrelationMethod','Season','Covid_Column','Temperature','Relative_Humidity','Multiple_Correlation'])
#corr_mul_TH.to_csv('gdrive/My Drive/Colab Notebooks/Research lab/Latest/results/complete_data/Multiple_Correlation.csv')
corr_mul_TH

,CorrelationMethod,Season,Covid_Column,Temperature,Relative_Humidity,Multiple_Correlation
0,Kendall-Tau,Total,PCR_Postive_Count,-0.0470,-0.2137,0.1324
1,Kendall-Tau,Total,PCR_Positive_Hospitalised,-0.0311,-0.0613,0.1195
2,Kendall-Tau,Total,Current_Infected,-0.1209,-0.2749,0.1680
3,Kendall-Tau,Total,Total_Confirmed,0.1174,-0.3562,0.1168
4,Spearmann,Total,PCR_Postive_Count,-0.0732,-0.3274,0.3128
5,Spearmann,Total,PCR_Positive_Hospitalised,-0.0420,-0.0831,0.2561
6,Spearmann,Total,Current_Infected,-0.1844,-0.4341,0.4503
7,Spearmann,Total,Total_Confirmed,0.1759,-0.5391,0.2619
